# <img src="./images/SQLIcon.png?modified=2334223" width=80px height=80px style="vertical-align: middle;">  SQL `JOIN`s

A SQL join is a method used to combine data from two or more tables in a relational database based on a common column/set of columns. The resulting combined table, called a *joined table*, contains columns and rows from both tables, where the join condition is satisfied in both tables. 

SQL `JOIN`'s are commonly used to combine related data from different tables into a single result set, allowing for more complex queries and analysis.

## Why use `JOIN`s?

So how can `JOIN`s help us, let's imagine we wanted to find all addresses that exist for a country in the pagila database. First let's view a `country`, for example `Argentina` in the `country` table and find all addresses for `Argentina` in the `address` table.


In [ ]:
SELECT country_id,
       country
FROM 
    country
WHERE 
    country = 'Argentina';

This gives us the result:

<img src='images/country_argentina.png?modified=34242'>

We now know that the `country_id` for `Argentina` is `6`. So we can search up the `city` table to get the respective cities and their `id`'s.

In [ ]:
SELECT country_id,
       city,
       city_id
FROM 
    city
WHERE 
    country_id = 6;

<img src='images/city_codes_argentina.png?modified=342423'>

Great this gives us all the cities and their `id`'s so we can now search all those codes in the `address` table. 

In [ ]:
SELECT address_id,
       address,
       address2,
       district,
       city_id,
       postal_code
FROM 
    address
WHERE city_id IN (20, 43, 45, 128, 161, 165, 289, 334, 424, 454, 457, 524, 567);

<img src='images/argentina_addresses.png?modified=233423432'>

Which gives us the correct result assuming we made no errors. You can see how tedious this process is. We needed to create three different queries to get the result we expect. 

There is also greater room for error since we've had to manually retrieve and input the `city_id`'s in the final statement. Imagine if there were over 30 cities in the country we were  interested in. It would be quite the task to manually input all those codes and we could easily mistype them.

This is where the `JOIN` keyword comes in, it allows us to join the tables together using some commonality between the tables to produce the data we require in one query. 

## The `JOIN` statement

<img src='images/four_types_joins.png?modified=1213232323'>

In SQL there are many different types of `JOIN`s you can use to get the result you want. The main four are mentioned above. All other `JOIN`'s which can be preformed in SQL will be some subset of the joins shown in the image. To create a `JOIN` in SQL using the following syntax: 

In [ ]:
{[INNER] | {LEFT | RIGHT | FULL} [OUTER]} JOIN {table2} ON {boolean condition};

First the statement starts with the type of `JOIN` you want to apply between tables, the options are:


- <b style="color: rgb(241, 90, 36)">INNER</b>: specifies to `JOIN` tables where records match in both. You can also have SQL perform an `INNER JOIN` by not specifying `INNER` OR `OUTER` in the join statement. Using `INNER JOIN` is just *syntactic sugar*(unnecessary syntax but makes code easier to read), so it's best practice to include it.
- <b style="color: rgb(241, 90, 36)">{LEFT | RIGHT | FULL} [OUTER]</b>: The other option is an `OUTER` join, where we also have to specify which table we want to target all the records of. The options here are `RIGHT`, `LEFT` or `FULL`. 
    - `RIGHT` specifies to target all records in the table on the right of the boolean condition (conversely for `LEFT`). 
    - `OUTER` targets records belonging to both tables. Examples of this we be explained later in the notebook. 

The statement is then followed by the `JOIN` keyword and the name of the table you want to join to. This is then followed by the `ON` keyword, which precedes what is known as the *join predicate*. The **join predicate** is a conditional statement which determines which records should be joined together. If the statement evaluates to `True` then the records will become part of the joined table results, otherwise they will be ignored. Multiple `JOIN` statements can be use together in conjunction with each other to get the data required. 

Let's see how the previous problem of getting all the Argentinian address from the `address` table can be more easily solved with joins. First we'll `JOIN` the `city` and `county` tables together:

In [ ]:
SELECT country.country_id ,
       country,
       city_id
FROM 
    country
INNER JOIN 
    city ON country.country_id = city.country_id
WHERE 
    country = 'Argentina';

So here we are performing an `INNER JOIN` on the `city` table. If the records match in both tables, then the record will be returned. For the join predicate we have stated `country.country_id = city.country_id`, we want to join the tables where the `country_id` matches in both tables. 

Notice in the join predicate we have had to define the columns to match in the form `country.country_id = city.country_id` this is due to the ambiguity of writing `city_id = city_id`. The column `city_id` exists in both tables so SQL won't know exactly which column you're referencing. We've also had to do this in the `SELECT` statement for the same reason. 

The form `{table_name.column_name}` is how you reference a specific column from a specific table. Which you will often have to do when performing joins since the data can exist in both tables. Note if tables or columns are capitalised then they would need to be wrapped in double quotes for SQL to understand which column you are defining. 

So now we have the result:

<img src="images/city_address_join_results.png?modified=3242342">

Which gives us the necessary `city_id`'s, we now have to join the address table to get the final results:

In [ ]:
SELECT address.country_id ,
       country,
       city.city_id,
       address,
       address2,
       postal_code,
       district
FROM 
    country
INNER JOIN 
    city ON country.country_id = city.country_id
INNER JOIN
    address ON address.city_id = city.city_id   
WHERE 
    country = 'Argentina';

We now add another `JOIN` statement to get the final results, we perform an `INNER JOIN` but this time joining `address`. Notice in this `JOIN` the `country` table isn't referenced in the join predicate. Instead we're referencing the `address` and `city` table. After the first `INNER JOIN` the `city` table was joined to the `country` table. So columns from `city` and the columns from `country` exist in a temporary SQL table until the query completes. So we can now join the `address` table referencing columns in the `city` table. If we reversed the order of the joins:

In [ ]:
FROM 
    country
INNER JOIN
    address ON address.city_id = city.city_id
INNER JOIN 
    city ON country.country_id = city.country_id

SQL will throw an error since we can't reference columns from the `city` table in the first `JOIN`, as they've not been joined to the `country` table yet. 

>A tip for resolving this is to build your `JOIN`s up in stages first. Perform the first `JOIN` and run the query to see what columns are now available to be joined. Then you have a clearer idea of what columns should be joined with your next `JOIN` statement. 

## Key Takeaways

- `JOIN`s are useful when you need data from multiple tables
- The order in which you perform your `JOIN`s matters
- Specify which column you want the tables to joined with using the **join predicate**
- Multiple `JOIN`s and of different types can be used together to get exactly the data you need